In [22]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
import pandas as pd
import numpy as np

import proyeccion_rdr.features as features
from proyeccion_rdr.produccion.a04_ambulatorio import (
    obtener_distribucion_consultas,
    procesar_incidencias,
    leer_y_filtrar_consultas_medicas_y_no_medicas,
    expandir_serie_rendimientos,
)

pd.set_option("display.max_columns", None)

ANIOS_POBLACION = [str(i) for i in range(2017, 2036)]

In [24]:
# Lee los casos de todos los macroprocesos
RUTA_ARCHIVOS = "../data/interim/0.1_casos_teoricos_diagnosticos.xlsx"
casos_macroproceso_por_region, casos_macroproceso = features.leer_casos_macroprocesos(
    RUTA_ARCHIVOS
)

# Pone el indice en el DataFrame de los casos
casos_macroproceso = casos_macroproceso.reset_index().set_index(["Diagnostico", "tipo_paciente"])
casos_macroproceso_por_region = casos_macroproceso_por_region.reset_index().set_index(
    ["Diagnostico", "tipo_paciente"]
)

# Define las areas de influencia presenciales
AREAS_PRESENCIALES = [
    "Metropolitana de Santiago",
    "COMUNAS_SIN_SS_EN_RM",
    "SSMC",
    "SSMN",
    "SSMO",
    "SSMOC",
    "SSMS",
    "SSMSO",
    "Acotado por oferta",
]
casos_macroproceso_por_region["es_presencial"] = np.where(
    casos_macroproceso_por_region["Estrato"].isin(AREAS_PRESENCIALES),
    "Box Presencial",
    "Box de Telemedicina",
)

## Objetivo de este cuadernillo

En este cuadernillo se quieren estimar la cantidad de pacientes ambulatorios que asisitrán al INT para el 2035. Este insumo se utilizará para estimar la cantidad de Box Ambulatorios para cada especialidad. Por lo tanto, es necesario hacer los siguientes calculos:

1. Pacientes Ambulatorios
2. Boxes Ambulatorios

In [27]:
RUTA_INCIDENCIAS = (
    "../data/raw/3_incidencias_y_porcentajes_marcoprocesos/incidencias_y_prevalencias_RDR.xlsx"
)

(
    df_incidencias,
    diagnosticos_ingresados,
    casos_por_especialidad_long,
    casos_totales_por_especialidad_y_grupo,
    casos_a_hacerse_cargo_long,
    casos_a_hacerse_cargo_por_especialidad_grupo_y_presencial,
    casos_a_hacerse_cargo_consolidados,
) = procesar_incidencias(
    RUTA_INCIDENCIAS, "consultas_medicas", casos_macroproceso_por_region, ANIOS_POBLACION
)

Iniciando proceso...
Cargando datos desde ../data/raw/3_incidencias_y_porcentajes_marcoprocesos/incidencias_y_prevalencias_RDR.xlsx, hoja: consultas_medicas
Separando diagnósticos por especialidad...
Diagnósticos únicos ingresados: 22
Indexando por tipo de paciente y diagnóstico...
Uniendo casos por diagnóstico con macroprocesos...
Identificando casos duplicados...
Especialidades con diagnósticos duplicados: 

ESTAMENTO/ESPECIALIDAD
PEDIATRÍA    [A080]
Name: Diagnostico, dtype: object

Sumando pacientes por grupo y especialidad...
Calculando casos a hacerse cargo...
Sumando pacientes por grupo y especialidad...
Sumando pacientes por grupo y especialidad...
Proceso completado.


## Test de Calidad

En este test se quiere saber si todas las trazadoras estan ingresadas en el macroproceso ambulatorio

In [28]:
# Lee la planilla de trazadoras y aisla
trazadoras_totales = pd.read_excel(RUTA_INCIDENCIAS, sheet_name="trazadoras")
trazadoras_totales = set(trazadoras_totales["Diagnostico"].str.split(" - ").str[0].unique())

In [29]:
diagnosticos_sin_ingresar = trazadoras_totales - diagnosticos_ingresados
if len(diagnosticos_sin_ingresar) > 0:
    raise ValueError(f"Falta ingresar las trazadoras: {diagnosticos_sin_ingresar}")

## Lectura de performance historico de ambulatorio

In [ ]:
# Lee las consultas medicas
RUTA_AMBULATORIO = "../data/raw/6_ambulatorio/df_procesada_consultas.csv"
_, consultas_medicas, _ = leer_y_filtrar_consultas_medicas_y_no_medicas(RUTA_AMBULATORIO)

In [34]:
import polars as pl

In [ ]:
df = pd.read_csv(RUTA_AMBULATORIO, dtype={"id_paciente": str})

In [62]:
df.groupby(["ano_ate", "unidada_ate_desc", "id_paciente"]).size().reset_index(name="n_consultas").groupby(["ano_ate", "unidada_ate_desc"])["n_consultas"].describe()

count      mean       std  min  25%  50%  \
ano_ate unidada_ate_desc                                                     
2017    ANESTESIOLOGIA INFANTIL   201.0  1.059701  0.341201  1.0  1.0  1.0   
        BRONCOPULMONAR INFANTIL  2917.0  2.682551  2.009753  1.0  1.0  2.0   
        CARDIOLOGIA INFANTIL     3377.0  2.276281  4.001008  1.0  1.0  2.0   
        CIRUGIA INFANTIL         3706.0  1.958716  1.452132  1.0  1.0  1.0   
        CIRUGIA PLASTICA          808.0  2.780941  2.440424  1.0  1.0  2.0   
...                                 ...       ...       ...  ...  ...  ...   
2024    PREMATUROS *              567.0  4.045855  2.548057  2.0  2.0  4.0   
        REUMATOLOGIA             1078.0  4.580705  3.688986  2.0  2.0  4.0   
        SALUD MENTAL             1037.0  5.402122  4.258686  2.0  2.0  4.0   
        TRAUMATOLOGIA INFANTIL   8394.0  3.716226  3.160130  2.0  2.0  2.0   
        UROLOGIA INFANTIL        1271.0  4.918961  4.494498  2.0  2.0  4.0   

                                 75%    max  
ano_ate unidada_ate_desc                     
2017    ANESTESIOLOGIA INFANTIL  1.0    5.0  
        BRONCOPULMONAR INFANTIL  4.0   25.0  
        CARDIOLOGIA INFANTIL     2.0  202.0  
        CIRUGIA INFANTIL         2.0   20.0  
        CIRUGIA PLASTICA         4.0   23.0  
...                              ...    ...  
2024    PREMATUROS *             6.0   20.0  
        REUMATOLOGIA             6.0   28.0  
        SALUD MENTAL             8.0   32.0  
        TRAUMATOLOGIA INFANTIL   4.0   58.0  
        UROLOGIA INFANTIL        6.0   36.0  

[229 rows x 8 columns]

In [ ]:
# Obtiene resumen de consultas acumuladas en el periodo
agrupacion_acumulada = ["especialidad_agrupada"]
distribucion_consultas_medicas_acumuladas, consultas_medicas_por_paciente_acumuladas = (
    obtener_distribucion_consultas(consultas_medicas, agrupacion_acumulada)
)

# Aisla el nombre de la columna que tenga el 75% de las consultas
columna_estadistica_cantidad_consultas = distribucion_consultas_medicas_acumuladas.columns[
    distribucion_consultas_medicas_acumuladas.columns.str.contains("75%")
][0]

# Indica cuantas consultas por cada una de las especialidades
cantidad_consultas_medicas_a_ocupar = distribucion_consultas_medicas_acumuladas[
    columna_estadistica_cantidad_consultas
]

# Filtra las especialidades para solo dejar las que estan en la cartera de servicios
cantidad_consultas_medicas_a_ocupar = cantidad_consultas_medicas_a_ocupar[
    cantidad_consultas_medicas_a_ocupar.index.isin(
        casos_a_hacerse_cargo_por_especialidad_grupo_y_presencial.index.get_level_values(0)
    )
]

In [ ]:
distribucion_consultas_medicas_acumuladas

,count_entre_2015_2024,mean_entre_2015_2024,std_entre_2015_2024,min_entre_2015_2024,25%_entre_2015_2024,50%_entre_2015_2024,75%_entre_2015_2024,max_entre_2015_2024,cantidad_consultas_entre_2015_2024
especialidad_agrupada,,,,,,,,,
CARDIOLOGÍA,11370.0,4.398505,5.765321,1.0,1.0,2.0,5.0,63.0,50011
CIRUGÍA CARDIOVASCULAR,9558.0,3.904373,3.773411,1.0,1.0,3.0,5.0,47.0,37318
CIRUGÍA DE TÓRAX,7226.0,3.272211,3.080591,1.0,1.0,2.0,4.0,38.0,23645
ENFERMEDADES RESPIRATORIAS ADULTO,29443.0,5.581259,7.416442,1.0,1.0,3.0,7.0,127.0,164329
Especialidad no ingresada,4088.0,2.697162,3.708853,1.0,1.0,1.0,3.0,62.0,11026
INFECTOLOGÍA,123.0,2.203252,1.783161,1.0,1.0,2.0,2.0,9.0,271
MEDICINA INTERNA,534.0,1.823970,1.366617,1.0,1.0,1.0,2.0,9.0,974
ONCOLOGÍA,3703.0,6.322441,8.660332,1.0,1.0,3.0,8.0,98.0,23412
PSIQUIATRÍA ADULTO,258.0,2.178295,1.942559,1.0,1.0,1.0,3.0,14.0,562


In [ ]:
cantidad_consultas_medicas_a_ocupar

especialidad_agrupada
CARDIOLOGÍA                          5.0
CIRUGÍA CARDIOVASCULAR               5.0
CIRUGÍA DE TÓRAX                     4.0
ENFERMEDADES RESPIRATORIAS ADULTO    7.0
INFECTOLOGÍA                         2.0
MEDICINA INTERNA                     2.0
ONCOLOGÍA                            8.0
PSIQUIATRÍA ADULTO                   3.0
Name: 75%_entre_2015_2024, dtype: float64

## Definicion de desempeno para consultas sin produccion

In [ ]:
# Define la cantidad de consultas que tendran las especialidades sin un rendimiento historico
consultas_sin_desempeno = pd.Series(
    {
        "ANESTESIOLOGÍA": 1,
        "GENÉTICA CLÍNICA": 3,
        "NUTRIOLOGO": 1,
        "ODONTOLOGÍA": 2,
        "PALIATIVISTA": 12,
        "RADIOTERAPEUTA": 2,
        "GASTROENTEROLOGIA": 1.5,
        "NEFROLOGÍA": 1.5,
        "NEUROLOGÍA": 1.5,
    }
)

# Concatena las especilidades que si tienen desempeno con las que no las tienen
cantidad_consultas_medicas_a_ocupar = pd.concat(
    [cantidad_consultas_medicas_a_ocupar, consultas_sin_desempeno]
)

# Cambia la cantidad de consultas que recibira Medicina Interna
cantidad_consultas_medicas_a_ocupar["MEDICINA INTERNA"] = 1.5
cantidad_consultas_medicas_a_ocupar.name = "consultas_medicas_a_ocupar"

In [ ]:
cantidad_consultas_medicas_a_ocupar

CARDIOLOGÍA                           5.0
CIRUGÍA CARDIOVASCULAR                5.0
CIRUGÍA DE TÓRAX                      4.0
ENFERMEDADES RESPIRATORIAS ADULTO     7.0
INFECTOLOGÍA                          2.0
MEDICINA INTERNA                      1.5
ONCOLOGÍA                             8.0
PSIQUIATRÍA ADULTO                    3.0
ANESTESIOLOGÍA                        1.0
GENÉTICA CLÍNICA                      3.0
NUTRIOLOGO                            1.0
ODONTOLOGÍA                           2.0
PALIATIVISTA                         12.0
RADIOTERAPEUTA                        2.0
GASTROENTEROLOGIA                     1.5
NEFROLOGÍA                            1.5
NEUROLOGÍA                            1.5
Name: consultas_medicas_a_ocupar, dtype: float64

In [ ]:
# Indica la cantidad de consultas que tendran la telemedicina
grupos_de_pacientes = [1, 2]
valores_es_presencial = ["Box Presencial", "Box de Telemedicina"]

# Expande la cantidad de consultas segun grupo de pacientes y valores presenciales
cantidad_consultas_medicas_a_ocupar = expandir_serie_rendimientos(
    cantidad_consultas_medicas_a_ocupar, grupos_de_pacientes, valores_es_presencial
)

In [ ]:
# Corrige la cantidad de consultas de pacientes paliativos en anestesiologia (A 6 consultas)
cantidad_consultas_medicas_a_ocupar[("ANESTESIOLOGÍA", 2, "Box Presencial")] = 6
cantidad_consultas_medicas_a_ocupar[("ANESTESIOLOGÍA", 2, "Box de Telemedicina")] = 6

In [ ]:
TIEMPO_CONSULTA = {
    "15 minutos": 0.25,
    "30 minutos": 0.5,
    "45 minutos": 0.75,
    "60 minutos": 1,
    "20 minutos": 0.333,
    "40 minutos": 0.666,
}

rendimientos_reales = pd.Series(
    {
        "CARDIOLOGÍA": TIEMPO_CONSULTA["30 minutos"],
        "CIRUGÍA CARDIOVASCULAR": TIEMPO_CONSULTA["30 minutos"],
        "CIRUGÍA DE TÓRAX": TIEMPO_CONSULTA["30 minutos"],
        "ENFERMEDADES RESPIRATORIAS ADULTO": TIEMPO_CONSULTA["20 minutos"],
        "INFECTOLOGÍA": TIEMPO_CONSULTA["20 minutos"],
        "MEDICINA INTERNA": TIEMPO_CONSULTA["60 minutos"],
        "ONCOLOGÍA": TIEMPO_CONSULTA["30 minutos"],
        "PSIQUIATRÍA ADULTO": TIEMPO_CONSULTA["45 minutos"],
        "ANESTESIOLOGÍA": TIEMPO_CONSULTA["20 minutos"],
        "GENÉTICA CLÍNICA": TIEMPO_CONSULTA["30 minutos"],
        "NUTRIOLOGO": TIEMPO_CONSULTA["20 minutos"],
        "ODONTOLOGÍA": TIEMPO_CONSULTA["45 minutos"],
        "PALIATIVISTA": TIEMPO_CONSULTA["40 minutos"],
        "RADIOTERAPEUTA": TIEMPO_CONSULTA["30 minutos"],
        "GASTROENTEROLOGIA": TIEMPO_CONSULTA["60 minutos"],
        "NEFROLOGÍA": TIEMPO_CONSULTA["60 minutos"],
        "NEUROLOGÍA": TIEMPO_CONSULTA["60 minutos"],
    }
)

# Expande los rendimientos
rendimientos_reales = expandir_serie_rendimientos(
    rendimientos_reales, grupos_de_pacientes, valores_es_presencial
)

In [ ]:
# Corrige rendimientos de pacientes paliativos en anestesiologia (A consulta de 40 minutos)
rendimientos_reales[("ANESTESIOLOGÍA", 2, "Box Presencial")] = 0.666
rendimientos_reales[("ANESTESIOLOGÍA", 2, "Box de Telemedicina")] = 0.666

## Estimacion de Consultas al 2035

In [ ]:
# Obtiene la cantidad de horas laborales
horas_laborales = build_features.calcular_horas_laborales(2017, 2035, 12)

Horas laborales por año calculadas:
+---+-------+--------------------------------------------+
|   | fecha | horas_laborales_funcionamiento_de_12_horas |
+---+-------+--------------------------------------------+
| 0 | 2017  |                    2976                    |
| 1 | 2018  |                    2964                    |
| 2 | 2019  |                    2988                    |
| 3 | 2020  |                    3024                    |
| 4 | 2021  |                    3012                    |
+---+-------+--------------------------------------------+



In [ ]:
# Multiplica los casos de area de influencia
consultas_proyectadas = casos_a_hacerse_cargo_por_especialidad_grupo_y_presencial.mul(
    cantidad_consultas_medicas_a_ocupar, axis=0
).dropna(axis=0, how="all")

# Multiplica la cantidad de consultas por el tiempo que requerira cada consulta
tiempo_consultas = consultas_proyectadas.mul(rendimientos_reales, axis=0).dropna(axis=0, how="all")

# Divide la cantidad de consultas por especialidad, por el rendimiento de cada box
boxes_proyectados = tiempo_consultas.div(horas_laborales, axis=1)

In [ ]:
# Obtiene el resumen de Boxes por estamnento
boxes_consolidados = (
    boxes_proyectados.reset_index()
    .groupby("ESTAMENTO/ESPECIALIDAD")
    .sum()["2035"]
    .sort_values(ascending=False)
)

# Agrega el total
boxes_consolidados["Total"] = boxes_consolidados.sum()

In [ ]:
display(boxes_consolidados)

ESTAMENTO/ESPECIALIDAD
ENFERMEDADES RESPIRATORIAS ADULTO     6.058044
CARDIOLOGÍA                           4.763821
ONCOLOGÍA                             3.597250
PALIATIVISTA                          3.234287
CIRUGÍA CARDIOVASCULAR                2.767801
ODONTOLOGÍA                           2.160367
MEDICINA INTERNA                      1.506224
GENÉTICA CLÍNICA                      1.348969
PSIQUIATRÍA ADULTO                    1.040690
ANESTESIOLOGÍA                        0.927789
CIRUGÍA DE TÓRAX                      0.691994
NUTRIOLOGO                            0.634506
RADIOTERAPEUTA                        0.569625
NEFROLOGÍA                            0.165739
INFECTOLOGÍA                          0.058403
NEUROLOGÍA                            0.029892
GASTROENTEROLOGIA                     0.027623
Total                                29.583024
Name: 2035, dtype: float64

In [ ]:
# Obtiene la cantidad de boxes medicos y de telemedicina
boxes_presencial_y_tele = (
    boxes_proyectados.reset_index().groupby("es_presencial")[ANIOS_POBLACION].sum()
)

In [ ]:
boxes_presencial_y_tele

,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035
es_presencial,,,,,,,,,,,,,,,,,,,
Box Presencial,18.518144,19.046577,19.405519,19.681692,20.050235,20.306822,20.706728,20.687721,20.832628,20.803336,20.933783,21.312656,21.612229,21.565977,21.520358,21.370992,21.639979,22.090796,22.285686
Box de Telemedicina,6.307187,6.382891,6.421172,6.440882,6.538711,6.622207,6.758309,6.759308,6.812390,6.808690,6.855210,6.982420,7.082177,7.066532,7.057236,7.008600,7.095238,7.239497,7.297338


In [ ]:
# Define columnas que se utilizar para generar resumen MINSAL
cols_indice = ["ESTAMENTO/ESPECIALIDAD", "Grupos de Pacientes", "es_presencial"]

# Genera un DataFrame de las trazadoras con presencialidad
df_incidencias_resumen = df_incidencias.copy()
df_incidencias_resumen["es_presencial"] = "Box Presencial, Box de Telemedicina"
df_incidencias_resumen["es_presencial"] = df_incidencias_resumen["es_presencial"].str.split(", ")
df_incidencias_resumen = df_incidencias_resumen.explode("es_presencial")
df_incidencias_resumen = df_incidencias_resumen.set_index(cols_indice)

## Resumen MINSAL

In [ ]:
# Obtiene el resumen MINSAL
resumen_MINSAL = pd.DataFrame(
    {
        "tipo_paciente": df_incidencias_resumen["tipo_paciente"],
        "Diagnostico": df_incidencias_resumen["Diagnostico"],
        "Areas de Influencia a atender presencial": str(AREAS_PRESENCIALES),
        "numero_de_pacientes_totales_2035": casos_totales_por_especialidad_y_grupo["2035"],
        "% de los pacientes a atender": df_incidencias_resumen["% de los pacientes a atender"],
        "explicacion_pacientes": df_incidencias_resumen["Explicación Pacientes"],
        "casos_a_hacerse_cargo_2035": casos_a_hacerse_cargo_por_especialidad_grupo_y_presencial[
            "2035"
        ],
        "consultas_por_paciente_75%": cantidad_consultas_medicas_a_ocupar,
        "consultas_proyectadas_2035": consultas_proyectadas["2035"],
        "horas_por_consultas": rendimientos_reales,
        "horas_consultas_2035": tiempo_consultas["2035"],
        "horas_laborales_2035": horas_laborales["2035"],
        "boxes_proyectados_2035": boxes_proyectados["2035"],
    }
)

resumen_MINSAL = resumen_MINSAL.dropna().reset_index(level=[1, 2])

In [ ]:
a_guardar = {
    "resumen_MINSAL": resumen_MINSAL,
    "boxes_consolidados": boxes_consolidados,
    "trazadoras_ambulatorio": df_incidencias,
    "casos_macroprocesos_por_region": casos_macroproceso_por_region.reset_index(),
    "casos_por_esp_long": casos_por_especialidad_long.reset_index(),
    "casos_a_hacerse_cargo_long": casos_a_hacerse_cargo_long,
    "casos_por_esp_grupo_y_pres": casos_a_hacerse_cargo_por_especialidad_grupo_y_presencial,
    "casos_consolidados": casos_a_hacerse_cargo_consolidados,
    "metricas_historicas_por_espec": distribucion_consultas_medicas_acumuladas,
    "rendimientos_utilizados": rendimientos_reales,
    "consultas_medicas_proyectadas": consultas_proyectadas,
    "tiempo_por_especialidad": tiempo_consultas,
    "horas_laborales": horas_laborales,
    "boxes_medicos_proyectados": boxes_proyectados,
    "boxes_presenciales_y_tele": boxes_presencial_y_tele,
}

with pd.ExcelWriter("../data/interim/3.0_estimacion_boxes_medicos_INT.xlsx") as file:
    for nombre_hoja, df_a_guardar in a_guardar.items():
        df_a_guardar.to_excel(file, sheet_name=nombre_hoja)